In [1]:
#Import libraries
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import TavilySearchTool, FileWriterTool
from dotenv import load_dotenv
from IPython.display import display, Markdown

In [2]:
# Load environment variables (TAVILY_API_KEY, LLM_GATEWAY_URL, etc.)
load_dotenv(override=True)

True

In [3]:
# --- Configuration ---
# 1. Update the LLM initialization for CrewAI with TrueFoundry
llm = LLM(
    model=os.getenv("LLM_MODEL_CREWAI"),
    base_url=os.getenv("LLM_GATEWAY_URL"),
    api_key=os.getenv("TFY_API_KEY")
)

In [4]:
# --- Tools Setup ---
# Tavily Search Tool for web research
tavily_search_tool = TavilySearchTool()

# File Writer Tool for saving the final report
file_writer_tool = FileWriterTool()

In [5]:
# --- 2. Agents Definition ---
# 1) Researcher Agent (Uses TavilySearchTool)
researcher_agent = Agent(
    role="Senior Web Researcher",
    goal="Gather the latest and most relevant information about the user's query and format it as a comprehensive summary.",
    backstory="""You are an expert at utilizing the Tavily web search tool to find real-time, accurate, 
                 and cited information on any given topic. Your output is precise and well-structured.""",
    tools=[tavily_search_tool],
    llm=llm,
    verbose=True,
    memory=False,
    allow_delegation=False
)

In [6]:
# 2) Report Agent (Uses FileWriterTool)
report_agent = Agent(
    role="Professional Technical Writer",
    goal="Write a final, professionally formatted markdown report based on the context provided by the Researcher Agent",
    backstory="""You are a meticulous technical writer who turns raw research data into polished, 
                 production-ready documentation.""",
    tools=[],
    llm=llm,
    verbose=True,
)

In [7]:
# --- 3. Tasks Definition ---
research_task = Task(
    description="""Conduct an 'advanced' web search for the user's query: '{query}'. 
                   Focus on recent developments (past 6 months) and list all sources.
                   The final output must be a single, well-structured text summary of findings.""",
    expected_output="A comprehensive, cited summary of the research topic.",
    agent=researcher_agent,
)

In [8]:
report_task = Task(
    description="""Based on the summary provided by the Researcher Agent, write a final report.
                   The report must be in **Markdown format** with a title and bullet points.""",
    expected_output="A Markdown formatted report with a title and bullet points.",
    agent=report_agent,
    context=[research_task] # Task waits for research to complete
)

In [9]:
# --- Crew Initialization ---
research_crew = Crew(
    agents=[researcher_agent, report_agent],
    tasks=[research_task, report_task],
    process=Process.sequential,
    verbose=True # Higher verbosity for clear demonstration
)

In [10]:
# --- Function to be Wrapped by FastMCP --- this is synchronous here, but we'll make it asynchronous in the final script.
def run_research_analysis(query: str):
    """Executes the two-agent research crew."""
    print(f"--- Starting Crew for query: {query} ---")
    
    # Pass the user query to the crew's initial task
    result = research_crew.kickoff(inputs={"query": query})
    
    # Return the final output message from the Report Agent
    return result

In [11]:
# Example Hackathon Query
query = "Latest developments in quantum computing hardware in 2025" 

final_output = run_research_analysis(query)

print("\n\n--- Final Result from Crew ---")
print(final_output)

--- Starting Crew for query: Latest developments in quantum computing hardware in 2025 ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7c89f55f-fc6d-4f93-85c8-4c86758c03a2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Web Researcher                                                                                   │
│                                                                                                                 │
│  Task: Conduct an 'advanced' web search for the user's query: 'Latest developments in quantum computing         │
│  hardware in 2025'.                                                                                             │
│                     Focus on recent developments (past 6 months) and list all sources.                          │
│                     The final output must be a single, well-structured text summary of findings.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Web Researcher                                                                                   │
│                                                                                                                 │
│  Thought: Thought: To provide a comprehensive and well-cited answer, I should perform a web search using the    │
│  Tavily Search tool to gather the latest information on quantum computing hardware developments in 2025.        │
│                                                                                                                 │
│  Using Tool: Tavily Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Technical Writer                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Quantum Computing Hardware in 2025: A Transformative Year                                                    │
│                                                                                                                 │
│  The year 2025 has been pivotal for the quantum computing industry, marked by substantial advancements in       │
│  hardware and collaborative breakthroughs. Key developments and milestones include:                             │
│                                                                                                                 │
│  - **Fujitsu and RIKEN Collaboration**:                                                                         │
│    - Achieved the development of a 256-qubit superconducting quantum computer.                                  │
│    - This is a significant scale-up, being four times larger than their 2023 version.                           │
│    - Demonstrates major progress in qubit scaling and superconducting technologies.                             │
│    - [Source: SpinQuanta                                                                                        │
│  News](https://www.spinquanta.com/news-detail/latest-quantum-computing-news-and-quantum-research)               │
│                                                                                                                 │
│  - **Quantum Hardware’s Practical and Scalable Advancements**:                                                  │
│    - Industries are increasingly focusing on harnessing the power of quantum computing.                         │
│    - Recent hardware breakthroughs mark 2025 as a crucial year for quantum applications.                        │
│    - [Source: SpinQ - Quantum Hardware                                                                          │
│  Explained](https://www.spinquanta.com/news-detail/quantum-hardware-explained-a-complete-guide)                 │
│                                                                                                                 │
│  - **IBM's Roadmap Towards Fault-Tolerant Quantum Computing**:                                                  │
│    - Introduction of a roadmap with successive versions of Nighthawk quantum processors planned from 2025 to    │
│  2028.                                                                                                          │
│    - These developments aim to facilitate the exploration of complex quantum circuits and advance software      │
│  development.                                                                                                   │
│    - [Source: IBM Quantum Blog](https://www.ibm.com/quantum/blog/large-scale-ftqc)                              │
│                                                                                                                 │
│  - **D-Wave Systems' Breakthroughs**:                                                                           │
│    - Demonstration of quantum computers outperforming classical supercomputers in specific complex tasks.       │
│    - Highlights the potential superiority of quantum computing in certain computational challenges.             │
│    - [Source: Constellation Research                                                                            │
│  Blog](https://www.constellationr.com/blog-news/insight

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8d60c019-3b63-4d04-8ccf-b653047863f1                                                                     │
│  Agent: Professional Technical Writer                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7c89f55f-fc6d-4f93-85c8-4c86758c03a2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Quantum Computing Hardware in 2025: A Transformative Year                                      │
│                                                                                                                 │
│  The year 2025 has been pivotal for the quantum computing industry, marked by substantial advancements in       │
│  hardware and collaborative breakthroughs. Key developments and milestones include:                             │
│                                                                                                                 │
│  - **Fujitsu and RIKEN Collaboration**:                                                                         │
│    - Achieved the development of a 256-qubit superconducting quantum computer.                                  │
│    - This is a significant scale-up, being four times larger than their 2023 version.                           │
│    - Demonstrates major progress in qubit scaling and superconducting technologies.                             │
│    - [Source: SpinQuanta                                                                                        │
│  News](https://www.spinquanta.com/news-detail/latest-quantum-computing-news-and-quantum-research)               │
│                                                                                                                 │
│  - **Quantum Hardware’s Practical and Scalable Advancements**:                                                  │
│    - Industries are increasingly focusing on harnessing the power of quantum computing.                         │
│    - Recent hardware breakthroughs mark 2025 as a crucial year for quantum applications.                        │
│    - [Source: SpinQ - Quantum Hardware                                                                          │
│  Explained](https://www.spinquanta.com/news-detail/quantum-hardware-explained-a-complete-guide)                 │
│                                                                                                                 │
│  - **IBM's Roadmap Towards Fault-Tolerant Quantum Computing**:                                                  │
│    - Introduction of a roadmap with successive versions of Nighthawk quantum processors planned from 2025 to    │
│  2028.                                                                                                          │
│    - These developments aim to facilitate the exploration of complex quantum circuits and advance software      │
│  development.                                                                                                   │
│    - [Source: IBM Quantum Blog](https://www.ibm.com/quantum/blog/large-scale-ftqc)                              │
│                                                                                                                 │
│  - **D-Wave Systems' Breakthroughs**:                                                                           │
│    - Demonstration of quantum computers outperforming classical supercomputers in specific complex tasks.       │
│    - Highlights the potential superiority of quantum computing in certain computational challenges.             │
│    - [Source: Constellation Research                  

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

--- Final Result from Crew ---
# Quantum Computing Hardware in 2025: A Transformative Year

The year 2025 has been pivotal for the quantum computing industry, marked by substantial advancements in hardware and collaborative breakthroughs. Key developments and milestones include:

- **Fujitsu and RIKEN Collaboration**: 
  - Achieved the development of a 256-qubit superconducting quantum computer.
  - This is a significant scale-up, being four times larger than their 2023 version.
  - Demonstrates major progress in qubit scaling and superconducting technologies.
  - [Source: SpinQuanta News](https://www.spinquanta.com/news-detail/latest-quantum-computing-news-and-quantum-research)

- **Quantum Hardware’s Practical and Scalable Advancements**:
  - Industries are increasingly focusing on harnessing the power of quantum computing.
  - Recent hardware breakthroughs mark 2025 as a crucial year for quantum applications.
  - [

In [12]:
#Final response
display(Markdown(final_output.raw))

# Quantum Computing Hardware in 2025: A Transformative Year

The year 2025 has been pivotal for the quantum computing industry, marked by substantial advancements in hardware and collaborative breakthroughs. Key developments and milestones include:

- **Fujitsu and RIKEN Collaboration**: 
  - Achieved the development of a 256-qubit superconducting quantum computer.
  - This is a significant scale-up, being four times larger than their 2023 version.
  - Demonstrates major progress in qubit scaling and superconducting technologies.
  - [Source: SpinQuanta News](https://www.spinquanta.com/news-detail/latest-quantum-computing-news-and-quantum-research)

- **Quantum Hardware’s Practical and Scalable Advancements**:
  - Industries are increasingly focusing on harnessing the power of quantum computing.
  - Recent hardware breakthroughs mark 2025 as a crucial year for quantum applications.
  - [Source: SpinQ - Quantum Hardware Explained](https://www.spinquanta.com/news-detail/quantum-hardware-explained-a-complete-guide)

- **IBM's Roadmap Towards Fault-Tolerant Quantum Computing**:
  - Introduction of a roadmap with successive versions of Nighthawk quantum processors planned from 2025 to 2028.
  - These developments aim to facilitate the exploration of complex quantum circuits and advance software development.
  - [Source: IBM Quantum Blog](https://www.ibm.com/quantum/blog/large-scale-ftqc)

- **D-Wave Systems' Breakthroughs**:
  - Demonstration of quantum computers outperforming classical supercomputers in specific complex tasks.
  - Highlights the potential superiority of quantum computing in certain computational challenges.
  - [Source: Constellation Research Blog](https://www.constellationr.com/blog-news/insights/2025-year-quantum-computing-already)

- **Overall Impact in 2025**:
  - The year marks a transformative phase, setting the foundation for unprecedented computational capabilities through enhanced qubit capacity, strategic initiatives, and groundbreaking research.
  - [Source: McKinsey - The Year of Quantum](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/the-year-of-quantum-from-concept-to-reality-in-2025)

These developments reflect a momentous leap forward in quantum computing hardware, promising enhanced computational power and new possibilities for various industries.